<a href="https://colab.research.google.com/github/DerekSHAOZH/FUB-22Win-Phylogeny-Inference-and-Application/blob/main/Exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1 (5 points)

## Add an attribute to your vertex class where you store DNA sequence. 
## Add a function to you tree class that reads a DNA sequence alignment from a  fasta file (https://en.wikipedia.org/wiki/FASTA_format), and stores the sequences as attributes in the vertex objects.

# Problem 2 (30 points)

## Write a function in your tree class that implements the two phases of the Sankoff algorithm. 15 points for each phase.

# Problem 3 (15 points)

## Use the following newick tree and cost/weight matrix for your analysis
$T = ((l_1,l_2),((l_3,l_4),l_5));$

$
  W =
  \left[ {\begin{array}{cccc}
    W_{A→A} & W_{A→T} & W_{A→G} & W_{A→C}\\
    W_{T→A} & W_{T→T} & W_{T→G} & W_{T→C}\\
    W_{G→A} & W_{G→T} & W_{G→G} & W_{G→C}\\
    W_{C→A} & W_{C→T} & W_{C→G} & W_{C→C}
  \end{array} } \right]
  = 
  \left[ {\begin{array}{cc}
    0 & 2 & 1 & 2 \\
    2 & 0 & 2 & 1 \\
    1 & 2 & 0 & 2 \\
    2 & 1 & 2 & 0 
  \end{array} } \right]
$

## a) (5 points) Add a function to you tree class that computes the number of parsimony informative sites. How many parsimony informative sites are there in the fasta file Sankoff.fas (https://classroom.google.com/c/NDkwOTk4NjQyMjQx/m/NTY5MjQ0MTY3NDI2/details)

## b) (5 points) What is the parsimony score for $T$?

## c) (5 points) What is an ancestral sequence at the root?

# Problem 1 (5 points)

## Add an attribute to your vertex class where you store DNA sequence. 
## Add a function to you tree class that reads a DNA sequence alignment from a  fasta file (https://en.wikipedia.org/wiki/FASTA_format), and stores the sequences as attributes in the vertex objects.

In [ ]:
# TODO: add to Tree class by incorporating into the "Set_basepair_leaves" function
def parse_FASTA_file(fasta_filename):
  with open(fasta_filename,'r') as fasta_file_contents:
    s = {}    #dictionary with sequence_ID as key, sequence as value
    for line in fasta_file_contents.readlines():
      if line.startswith('>'): # new sequence
        sequence_ID = ((line.split(">")[1]).split("_")[0] + (line.split(">")[1]).split("_")[1]).replace("\n", "") #same leaf name as in newick form
      #elif line.startswith(';'):
      #  continue
      else:
        s[sequence_ID] = line[0:len(line)-1] #len(line)-1 to get rid of the "\n"
    return s

In [ ]:
parse_FASTA_file("/content/sankoff.fas")

{'l1': 'AACCGATTAGTAAGGAGCCCCATCGCCTGTTCTTACCAGTACGGGACCCATCACATTTTGGGTGGCGGTACGTGCTGCCCGCAGACAGTCGAAAAATACGTCAAGCCTGCAATTATTACAACCCAACACGGCTATCGGGCTTTCTGGAAGTTCGCCCGAAGTTGAAGTGCCGACCTTTGCTATACAACTAATCTTCGCCACGACCCATTTAGCATCTACGGGCCATCGAAGAGCCTACCCTTCAGACGTAAAACCAACACGTCGAAGGCATTACATTGTGCAACCATGCCCACATATCAACTTCGCCGTAGGCAGACATGAGATATCTAGCCCTACGGAAAGAGTCTTCGGGAGCAATGCCGCCCTAGGTCTATGCCCATATCATACAAGCTTGATGTTGGGTTCGCTCTGCCTTGCTTCCCCGCGATCCAGCCACTTCTGGTCCTATGCCGGATATACAAATAACTTTATATTAGATTATTACCTACTTAAGGCGGTCAAGGATTTTCGAAGGAATAAGGCAGTCTATTGAGTATGACACTGTCCTTGCAGTGTGCCACCCAACCTCGCTCCGGCCTTGTTAATCTTTCAGCTAGAGATATCGACTTCCACCTGCTCTATAAACATAACCCTGCTCTGGCTTGTCGGTGGGCGTTGCTTCGCTGCTACGAACTGTCAACATGCCAGGATTTATTTTGGGGTGATGCTATAACCCCGTTCCAAACACGACTTGGGCCGGTTTTTTTACTAGATTTGACACTAGGACATTTTGCCCTCCTTTTCCCATATCTCGTAAGTTAAGGGACTAGCTTCTAACGATTACCGTAATAACACGAACCCGGCGAGCAACTCCGCCTAAGATAATCAAAATGAGGAACTATTGGCAGTAAGGAGTCCTTCAATATCGTTCTTTAAACGTGGGTGGACAGTTATCAGGCACGAGCGTCGCGAGGATGTGCTGACGACCAAACAATTTCAACTTTCCGGTTCTC

# Problem 2 (30 points)

## Write a function in your tree class that implements the two phases of the Sankoff algorithm. 15 points for each phase.

*Added to the exercise 1 solution code that has been copied below

In [ ]:
#weight matrix for Sankoff algorithm
W = [[0, 2, 1, 2],
     [2, 0, 2, 1],
     [1, 2, 0, 2],
     [2, 1, 2, 0]]

In [ ]:
#helper function for phase 2 of Sankoff algorithm
def get_minindex(inputlist):
 
    #get the minimum value in the list
    min_value = min(inputlist)
 
    #return the index of minimum value 
    min_index = inputlist.index(min_value)
    return min_index


#dictionary encoding A as 0, C as 1, G as 2, T as 3, and vice versa, for phase 2 of Sankoff algorithm
bp_to_num = {'A':0, 'C':1, 'G':2, 'T':3}
num_to_bp = {0:'A', 1:'C', 2:'G', 3:'T'}

In [ ]:
import re
# 1 A and B class definitions and functions
inf = float('inf')

class Vertex:
  def __init__(self,name): 
    self.name = name
    self.in_degree = 0
    self.out_degree = 0
    self.parent = self
    self.children = []
    self.neighbors = []
    self.newick_label = ""
    self.base_pair = "" #store the sequence
    self.costs = [0]*4  #record the costs 

class Tree:
  def __init__(self,name):
    self.name = name
    self.vertex_map = {}
    self.leaf_list = []
    self.pre_order_list = []
    self.post_order_list = []
    self.edge_list_map = {}
    self.root = -1
    self.total_parsimony = 0  #store the total parsimony scores given sequences
  def Add_vertex(self,name):    
    v = Vertex(name)
    self.vertex_map[name] = v  
  def Contains_vertex(self,name):
    return (name in self.vertex_map.keys())
  def Get_vertex(self,name):
    if name in self.vertex_map.keys():
      return (self.vertex_map[name])
  def Add_directed_edge(self, parent_name, child_name, distance = 0):     #Default branch length set to 0 for Problem 3
    if parent_name not in self.vertex_map.keys():
      self.Add_vertex(parent_name)
    if child_name not in self.vertex_map.keys():
      self.Add_vertex(child_name)
    p = self.Get_vertex(parent_name)
    c = self.Get_vertex(child_name)
    p.out_degree += 1
    c.in_degree += 1
    c.parent = p
    p.children.append(c)
    self.edge_list_map[(p,c)] = distance
  def Get_edge_length(self, parent, child):
    return (self.edge_list_map[(parent, child)])
  def Set_root(self):
    for vertex in self.vertex_map.values():
      if vertex.in_degree == 0:
        self.root = vertex
  def Get_root(self):
    if self.root == -1:
      self.Set_root()
    return (self.root)
  def Set_post_order(self):
    self.post_order_list = [self.root]
    vertices_to_visit = [self.root]
    while len(vertices_to_visit) > 0:
      v = vertices_to_visit.pop()
      vertices_to_visit += v.children      
      self.post_order_list = v.children + self.post_order_list
  def Set_pre_order(self, root):      #pre-order traversal 
      if root:
          self.pre_order_list.append(root)
          if root.out_degree != 0:
            self.Set_pre_order(root.children[0])  #recursion
            self.Set_pre_order(root.children[1])  #recursion
  def Set_basepair_leaves(self, fasta_filename):  #make use of the function defined in Problem 1
    leaf_bp_dict = parse_FASTA_file(fasta_filename)
    for vertex in self.vertex_map.values():
      if vertex.name in leaf_bp_dict.keys():
        vertex.base_pair = leaf_bp_dict[vertex.name]
        print(f'SET {vertex.name}: {vertex.base_pair}')
  def Compute_newick_format(self):
    if len(self.post_order_list) != len(self.vertex_map):
      self.Set_post_order()
    for v in self.post_order_list:
      if v.out_degree == 0:
        v.newick_label = v.name
      else:        
        c_l = v.children[0]
        c_r = v.children[1]        
        len_l = self.Get_edge_length(v,c_l)
        len_r = self.Get_edge_length(v,c_r)
        v.newick_label = f'({c_l.newick_label}:{len_l},{c_r.newick_label}:{len_r})'
    self.root.newick_label += ";"
    return(self.root.newick_label)    
  def Read_newick_string(self, newick_string):    
    rx = r'\([^()]+\)'
    hidden_vertex_ind = 1
    while "(" in newick_string:                  
      # search for the parenthesis
      m = re.search(rx,newick_string)
      # returns a tuple containing all the subgroups of the match "()"
      string_match = m.group()            
      # remove ( and )
      siblings_string = string_match[1:-1]
      c_left_name_and_length, c_right_name_and_length = siblings_string.split(",")
      c_left_name, c_left_length = c_left_name_and_length.split(":")
      c_right_name, c_right_length = c_right_name_and_length.split(":")
      if not self.Contains_vertex(c_left_name):
          self.Add_vertex(c_left_name)
      if not self.Contains_vertex(c_right_name):
          self.Add_vertex(c_right_name)
      hidden_vertex_name = "h" + str(hidden_vertex_ind)
      self.Add_vertex(hidden_vertex_name)            
      self.Add_directed_edge(hidden_vertex_name, c_left_name, float(c_left_length))
      self.Add_directed_edge(hidden_vertex_name, c_right_name, float(c_right_length))
      newick_string = newick_string.replace(string_match,hidden_vertex_name)
      hidden_vertex_ind += 1 

  def Read_newick_string_without_length(self, newick_string):    #Read the newich form without branch length specified in Problem 3
    rx = r'\([^()]+\)'
    hidden_vertex_ind = 1
    while "(" in newick_string:                  
      # search for the parenthesis
      m = re.search(rx,newick_string)
      # returns a tuple containing all the subgroups of the match "()"
      string_match = m.group()            
      # remove ( and )
      siblings_string = string_match[1:-1]
      
      c_left_name, c_right_name = siblings_string.split(",")

      if not self.Contains_vertex(c_left_name):
          self.Add_vertex(c_left_name)
      if not self.Contains_vertex(c_right_name):
          self.Add_vertex(c_right_name)
      hidden_vertex_name = "h" + str(hidden_vertex_ind)
      self.Add_vertex(hidden_vertex_name)            
      
      self.Add_directed_edge(hidden_vertex_name, c_left_name)

      self.Add_directed_edge(hidden_vertex_name, c_right_name)
      newick_string = newick_string.replace(string_match,hidden_vertex_name)
      hidden_vertex_ind += 1  

  def Num_informative(self):  ###Problem 3(a)
    sequences = []
    for vertex in self.vertex_map.values():
      if vertex.out_degree == 0:
        sequences.append(vertex.base_pair)  #gather aligned sequences in a list
  
    bool_informative = [0]*len(sequences[0]) #a boolean list with 1 indicating informative
    for i in range(len(sequences[0])):       #iterate over the length of sequence
      freq = {'A':0, 'T':0, 'C':0, 'G':0}    #keep track of the frequency of each bp in sequences
      for j in range(len(sequences)):
        freq[sequences[j][i]] += 1
      if sum(i >= 2 for i in freq.values()) >= 2: # count # of frequencies that are larger than 2
        bool_informative[i] = 1 # if # > 2, then record i
  
    num_informative = sum(bool_informative)  
    return num_informative

  def sankOff(self, W):          ###Problem 2

    #get the length of given sequences
    seq_length = 0
    for v in self.vertex_map.values():
      if v.out_degree == 0:
        seq_length = len(v.base_pair)
        break
        
    self.Set_root()
    self.Set_post_order()
    self.Set_pre_order(self.Get_root())

    leaf_costs = {
      'A': [0, inf, inf, inf],
      'C': [inf, 0, inf, inf],
      'G': [inf, inf, 0, inf],
      'T': [inf, inf, inf, 0],
    } 

    #iterate over the length of sequence to calculate parsimony for each bp
    for k in range(seq_length):

      #Phase 1
      for v in self.post_order_list:
        if len(v.children) == 0: #if vertex is a leaf
          leaf_cost = leaf_costs[v.base_pair[k]]
          v.costs = leaf_cost  #then we assign [0 inf] to leaves
        else:
          v.costs = [0]*4      #re-initiate the cost list after each iteration

          for i in range(0, 4):
            for c in v.children:
              children_plus_W = [sum(x) for x in zip(c.costs, W[i])]   
              child_cost = min(children_plus_W)
              v.costs[i] += child_cost
          
      #Phase 2
      for v in self.pre_order_list:
        if v.out_degree != 0: #if v is hidden vertex
          if v.in_degree == 0:  #if v is root
            v.base_pair += num_to_bp[get_minindex(v.costs)]        #in case of k-length sequences
            # v.base_pair = num_to_bp[get_minindex(v.costs)]          #in case of single base pair
            T.total_parsimony += min(v.costs)

          else: #if v is internal hidden vertex
            p = v.parent
            p_bp = v.parent.base_pair[k]        #in case of k-length sequences
            # p_bp = v.parent.base_pair            #in case of single base pair
            parent_plus_W = [sum(x) for x in zip(v.costs, W[bp_to_num[p_bp]])]
            v.base_pair += num_to_bp[get_minindex(parent_plus_W)]        #in case of k-length sequences
            # v.base_pair = num_to_bp[get_minindex(parent_plus_W)]          #in case of single base pair



## Sankoff algorithm: phase 1

_Compute costs for subtrees in postorder_

$$S_i(x) = \begin{cases}
[0, \infty, \infty, \infty] &  \\
\min_y[W_{x \to y} + S_j(y)] + \min_z[W_{x \to z} + S_k(z)] & 
\end{cases}$$

if $i$ is a leaf and $A$ is observed, where $j$ and $k$ are children of $i$

## Sankoff algorithm: phase 2

Assign ancestral characters $C_i$ in preorder such that total mutation cost is minimized

$$C_i = \begin{cases}
\min_x[S_i(x)] & \text{ if } i \text{ is the root} \\
\min_y[W_{x \to y} + S_i(y)] & \text{ where } x \text{ is the character in the parent of } i
\end{cases}$$

#Problem 3
##a) (5 points) Add a function to you tree class that computes the number of parsimony informative sites. How many parsimony informative sites are there in the fasta file Sankoff.fas (https://classroom.google.com/c/NDkwOTk4NjQyMjQx/m/NTY5MjQ0MTY3NDI2/details)


In [ ]:
T = Tree("T")
T.Read_newick_string_without_length('((l1,l2),((l3,l4),l5));')

for v in T.vertex_map.values():
  print (f'vertex {v.name} has in-degree {v.in_degree} and out-degree {v.out_degree}')



vertex l1 has in-degree 1 and out-degree 0
vertex l2 has in-degree 1 and out-degree 0
vertex h1 has in-degree 1 and out-degree 2
vertex l3 has in-degree 1 and out-degree 0
vertex l4 has in-degree 1 and out-degree 0
vertex h2 has in-degree 1 and out-degree 2
vertex l5 has in-degree 1 and out-degree 0
vertex h3 has in-degree 1 and out-degree 2
vertex h4 has in-degree 0 and out-degree 2


In [ ]:
T.Set_basepair_leaves('sankoff.fas')

for v in T.vertex_map.values():
  print (f'{v.name} has a sequence {v.base_pair}')


SET l1: AACCGATTAGTAAGGAGCCCCATCGCCTGTTCTTACCAGTACGGGACCCATCACATTTTGGGTGGCGGTACGTGCTGCCCGCAGACAGTCGAAAAATACGTCAAGCCTGCAATTATTACAACCCAACACGGCTATCGGGCTTTCTGGAAGTTCGCCCGAAGTTGAAGTGCCGACCTTTGCTATACAACTAATCTTCGCCACGACCCATTTAGCATCTACGGGCCATCGAAGAGCCTACCCTTCAGACGTAAAACCAACACGTCGAAGGCATTACATTGTGCAACCATGCCCACATATCAACTTCGCCGTAGGCAGACATGAGATATCTAGCCCTACGGAAAGAGTCTTCGGGAGCAATGCCGCCCTAGGTCTATGCCCATATCATACAAGCTTGATGTTGGGTTCGCTCTGCCTTGCTTCCCCGCGATCCAGCCACTTCTGGTCCTATGCCGGATATACAAATAACTTTATATTAGATTATTACCTACTTAAGGCGGTCAAGGATTTTCGAAGGAATAAGGCAGTCTATTGAGTATGACACTGTCCTTGCAGTGTGCCACCCAACCTCGCTCCGGCCTTGTTAATCTTTCAGCTAGAGATATCGACTTCCACCTGCTCTATAAACATAACCCTGCTCTGGCTTGTCGGTGGGCGTTGCTTCGCTGCTACGAACTGTCAACATGCCAGGATTTATTTTGGGGTGATGCTATAACCCCGTTCCAAACACGACTTGGGCCGGTTTTTTTACTAGATTTGACACTAGGACATTTTGCCCTCCTTTTCCCATATCTCGTAAGTTAAGGGACTAGCTTCTAACGATTACCGTAATAACACGAACCCGGCGAGCAACTCCGCCTAAGATAATCAAAATGAGGAACTATTGGCAGTAAGGAGTCCTTCAATATCGTTCTTTAAACGTGGGTGGACAGTTATCAGGCACGAGCGTCGCGAGGATGTGCTGACGACCAAACAATTTCAACTTTCCGGTTCTC

In [ ]:
T.sankOff(W)

In [ ]:
for v in T.vertex_map.values():
  print (f'{v.name} has a sequence {v.base_pair}')


l1 has a sequence AACCGATTAGTAAGGAGCCCCATCGCCTGTTCTTACCAGTACGGGACCCATCACATTTTGGGTGGCGGTACGTGCTGCCCGCAGACAGTCGAAAAATACGTCAAGCCTGCAATTATTACAACCCAACACGGCTATCGGGCTTTCTGGAAGTTCGCCCGAAGTTGAAGTGCCGACCTTTGCTATACAACTAATCTTCGCCACGACCCATTTAGCATCTACGGGCCATCGAAGAGCCTACCCTTCAGACGTAAAACCAACACGTCGAAGGCATTACATTGTGCAACCATGCCCACATATCAACTTCGCCGTAGGCAGACATGAGATATCTAGCCCTACGGAAAGAGTCTTCGGGAGCAATGCCGCCCTAGGTCTATGCCCATATCATACAAGCTTGATGTTGGGTTCGCTCTGCCTTGCTTCCCCGCGATCCAGCCACTTCTGGTCCTATGCCGGATATACAAATAACTTTATATTAGATTATTACCTACTTAAGGCGGTCAAGGATTTTCGAAGGAATAAGGCAGTCTATTGAGTATGACACTGTCCTTGCAGTGTGCCACCCAACCTCGCTCCGGCCTTGTTAATCTTTCAGCTAGAGATATCGACTTCCACCTGCTCTATAAACATAACCCTGCTCTGGCTTGTCGGTGGGCGTTGCTTCGCTGCTACGAACTGTCAACATGCCAGGATTTATTTTGGGGTGATGCTATAACCCCGTTCCAAACACGACTTGGGCCGGTTTTTTTACTAGATTTGACACTAGGACATTTTGCCCTCCTTTTCCCATATCTCGTAAGTTAAGGGACTAGCTTCTAACGATTACCGTAATAACACGAACCCGGCGAGCAACTCCGCCTAAGATAATCAAAATGAGGAACTATTGGCAGTAAGGAGTCCTTCAATATCGTTCTTTAAACGTGGGTGGACAGTTATCAGGCACGAGCGTCGCGAGGATGTGCTGACGACCAAACAATTTCAACTT

In [ ]:
print(f'Number of parsimony informative sites: {T.Num_informative()}')

Number of parsimony informative sites: 53


## b) (5 points) What is the parsimony score for $T$?



In [ ]:
print(f'The parsimony score for T is {T.total_parsimony}')

The parsimony score for T is 224


## c) (5 points) What is an ancestral sequence at the root?

In [ ]:
print(f'The ancestral sequence at root is {T.Get_root().base_pair}')

The ancestral sequence at root is AACCGAGTGGTAAGGAGCCCCATCGCCTGTTCTTACCAGTACGGGACCCATCACATTTTGGGTGGCAGTACGTGCTGCCCGCAGACAGTCGAAAAATACCTCAAGCCCGCAATTATTACAACCCAACACAGCTATCGGGCTTTCTGGAAGTTCGCCCGAAGTTCAGCTGCCGACCAATGCTATACAACTAATCTTCGACACGACCCATTTAGCATCTACGGGCCATCGAAGAGCCTACCCTTCAGACGTAAAACCAACACGTCGAAGGCATTACATTGTGCAACCATGCCCACATATCAACTTCGCCGTAGGCAGACATGAGATATCTAGCCCTACGGAAAGAGTCTTAGGGAGCAATGCCGCCCTAGGTCTATGCCCATATCATACAAGCTTGATGTTGGGTTCGCTCTGCCTTGCTTCCCCGCGATCCAGCCACTTCTGGTCCTATGCCGGATATACAAATAACTTTATATTAGATTATTACCTACTTAAGGCGCTCAAGGATATTCGAAGGATTAAGGCAGTCTAGTGAGTATGACACTGTCCCTGCAGTGTGCCACCCAACCTCGCGCCGGCCTTGTTAATCTTTCAGCTAGAGGTATCGACTTCCACCTGCTCTATAAGCATAACCCTGCTCTGGCTTGTCGGTGGGCGTTGCTTCGCTGATACGAACTGTCAACATGCCAGGATTTATTTTGGGGTGATGCTATAACCCCGTTCCAAACACGACTTGGGCCGGGTTTTTTACTAGATTTGACACTACGACATTTTGCCCTCCTTATCCCATATCTCGTAATTTAAGGGACTAGCTTCTAACGATGACCGTAATAACACGAACCCGGCGAGCAACTCCGACTAAGATAATCATAATCAGTAACTATTGGCAGTAAGGAGTCCTACAATATCGTTCTCTAAACGCCGGTGGACAGTGATCAGGCACAAGCGTCGCGAGGATGCGCTGACGAC